## Prédiction du Prix au m² à Lille

###  Objectif

Évaluer différents modèles de machine learning pour prédire le **prix au m²** de biens immobiliers à Lille, en séparant l'analyse entre **maisons** et **appartements**.

###  Import des librairies

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import joblib
import pickle
import os


### Chargement et filtrage

In [2]:

# 1. Chargement des données
df = pd.read_csv('../data/lille_2022.csv')

# 2. Filtrer uniquement les biens avec 4 pièces
df = df[df['Nombre pieces principales'] == 4]


### Séparation appartements/maisons + sélection des colonnes

In [3]:

# 3. Séparer appartements et maisons
df_appart = df[df['Type local'] == 'Appartement'].copy()
df_maison = df[df['Type local'] == 'Maison'].copy()

# 4. Colonnes utiles
colonnes_utiles = [
    'Surface reelle bati',
    'Nombre pieces principales',
    'Type local',
    'Surface terrain',
    'Nombre de lots',
    'Valeur fonciere'
]
df_appart = df_appart[colonnes_utiles].copy()
df_maison = df_maison[colonnes_utiles].copy()


### Nettoyage et création des variables

- Nettoyage des données
- Création de la variable cible prix_m2
- Suppression des outliers via la méthode IQR (Interquartile Range)

In [4]:

# # 5. 🧼 Traitement des valeurs manquantes
# df_appart['Surface terrain'] = df_appart['Surface terrain'].fillna(0)
# df_maison['Surface terrain'] = df_maison['Surface terrain'].fillna(df_maison['Surface terrain'].median())

# 6. Nettoyage ciblé
df_appart.dropna(subset=colonnes_utiles, inplace=True)
df_maison.dropna(subset=colonnes_utiles, inplace=True)

# 7. Création de la variable cible
df_appart['prix_m2'] = df_appart['Valeur fonciere'] / df_appart['Surface reelle bati']
df_maison['prix_m2'] = df_maison['Valeur fonciere'] / df_maison['Surface reelle bati']

# 8. Suppression des valeurs aberrantes via IQR
def clean_outliers_iqr(df, colonne='prix_m2'):
    Q1 = df[colonne].quantile(0.25)
    Q3 = df[colonne].quantile(0.75)
    IQR = Q3 - Q1
    borne_inf = Q1 - 1.5 * IQR
    borne_sup = Q3 + 1.5 * IQR
    return df[(df[colonne] >= borne_inf) & (df[colonne] <= borne_sup)]

df_appart = clean_outliers_iqr(df_appart)
df_maison = clean_outliers_iqr(df_maison)

print(f'Appartements après nettoyage : {len(df_appart)}')
print(f'Maisons après nettoyage : {len(df_maison)}')


Appartements après nettoyage : 49
Maisons après nettoyage : 326


### Fonction d’entraînement et sélection de modèles
Contient une grosse fonction modele_pipeline() qui :

- Sépare X (features) et y (cible)
- Applique une standardisation
- Entraîne plusieurs modèles : LinearRegression, DecisionTree, RandomForest, XGBoost
- Optimise DecisionTree et RandomForest avec GridSearchCV
- Retourne un DataFrame des résultats (MSE, RMSE, R²)
- Enfin, un tableau comparatif des performances est créé.

In [5]:

# ================================
# 1. Fonction d'entraînement avec encodage Type local
# ================================
def modele_pipeline(df, label, save_scaler_prefix=None):
    # Colonnes numériques
    X_num = df[['Surface reelle bati', 'Nombre de lots', 'Surface terrain']]
    # Encodage one-hot de 'Type local' (ex: Maison = 1, Appartement = 0)
    X_cat = pd.get_dummies(df['Type local'], drop_first=True)
    # Concaténation
    X = pd.concat([X_num, X_cat], axis=1)

    y = df['prix_m2']

    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

    # --- SCALING ---
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_test_scaled = scaler_X.transform(X_test)
    y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()
    y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).ravel()

    # Sauvegarde des scalers si demandé
    if save_scaler_prefix is not None:
        joblib.dump(scaler_X, f"../models/scaler_X_{save_scaler_prefix}.pkl")
        joblib.dump(scaler_y, f"../models/scaler_y_{save_scaler_prefix}.pkl")

    models = {
        'LinearRegression': LinearRegression(),
        'DecisionTree': DecisionTreeRegressor(random_state=42),
        'RandomForest': RandomForestRegressor(random_state=42)
    }

    results = {}
    trained_models = {}

    for name, model in models.items():
        model.fit(X_train_scaled, y_train_scaled)
        y_pred = model.predict(X_test_scaled)
        mse = mean_squared_error(y_test_scaled, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test_scaled, y_pred)
        results[name] = {'MSE': mse, 'RMSE': rmse, 'R2': r2}
        trained_models[name] = model

    # DecisionTree Optimisé
    tree_params = {'max_depth': [2, 4, 6, 8, 10]}
    grid_tree = GridSearchCV(DecisionTreeRegressor(random_state=42), tree_params, cv=5)
    grid_tree.fit(X_train_scaled, y_train_scaled)
    best_tree = grid_tree.best_estimator_
    y_pred = best_tree.predict(X_test_scaled)
    results['DecisionTree_Optimized'] = {
        'MSE': mean_squared_error(y_test_scaled, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test_scaled, y_pred)),
        'R2': r2_score(y_test_scaled, y_pred)
    }
    trained_models['DecisionTree_Optimized'] = best_tree

    # RandomForest Optimisé
    rf_params = {'n_estimators': [50, 100], 'max_depth': [None, 10, 20]}
    grid_rf = GridSearchCV(RandomForestRegressor(random_state=42), rf_params, cv=5)
    grid_rf.fit(X_train_scaled, y_train_scaled)
    best_rf = grid_rf.best_estimator_
    y_pred = best_rf.predict(X_test_scaled)
    results['RandomForest_Optimized'] = {
        'MSE': mean_squared_error(y_test_scaled, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test_scaled, y_pred)),
        'R2': r2_score(y_test_scaled, y_pred)
    }
    trained_models['RandomForest_Optimized'] = best_rf

    # XGBoost
    xgb = XGBRegressor(random_state=42, verbosity=0)
    xgb.fit(X_train_scaled, y_train_scaled)
    y_pred = xgb.predict(X_test_scaled)
    results['XGBoost'] = {
        'MSE': mean_squared_error(y_test_scaled, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test_scaled, y_pred)),
        'R2': r2_score(y_test_scaled, y_pred)
    }
    trained_models['XGBoost'] = xgb

    # Résumé en DataFrame
    df_resultats = pd.DataFrame(results).T
    df_resultats = df_resultats[['MSE', 'RMSE', 'R2']]
    df_resultats.index.name = 'Modèle'
    df_resultats['Type'] = label.split(' - ')[0]  # Exemple: \"Appartements\" ou \"Maisons\" ou \"Global\"

    return results, trained_models, df_resultats

# Sinon, entraînement séparé :
resultats_appart, models_appart, df_appart_resultats = modele_pipeline(df_appart, "Appartements - Lille", save_scaler_prefix="appart")
resultats_maison, models_maison, df_maison_resultats = modele_pipeline(df_maison, "Maisons - Lille", save_scaler_prefix="maison")

# ================================
# 3. Comparatif global
# ================================
df_comparatif = pd.concat([df_appart_resultats, df_maison_resultats])
df_comparatif = df_comparatif[['Type', 'MSE', 'RMSE', 'R2']]
df_comparatif_sorted = df_comparatif.sort_values(by=['Type', 'R2'], ascending=[True, False])

print("\n📊 Comparatif global des performances des modèles (Appartements & Maisons) :\n")
print(df_comparatif_sorted.round(3))



📊 Comparatif global des performances des modèles (Appartements & Maisons) :

                                Type    MSE   RMSE     R2
Modèle                                                   
RandomForest            Appartements  0.216  0.464  0.731
RandomForest_Optimized  Appartements  0.216  0.464  0.731
XGBoost                 Appartements  0.317  0.563  0.605
DecisionTree            Appartements  0.454  0.674  0.434
DecisionTree_Optimized  Appartements  0.633  0.796  0.212
LinearRegression        Appartements  0.867  0.931 -0.080
LinearRegression             Maisons  0.865  0.930  0.022
DecisionTree_Optimized       Maisons  1.020  1.010 -0.153
RandomForest_Optimized       Maisons  1.071  1.035 -0.210
RandomForest                 Maisons  1.140  1.068 -0.289
XGBoost                      Maisons  1.412  1.188 -0.596
DecisionTree                 Maisons  2.171  1.473 -1.454


###  Analyse de la Prédiction du Prix au m² à Lille


Le **meilleur modèle** pour la prédiction du **prix au m² à Lille** est celui qui présente :

- le **plus petit MSE** (Mean Squared Error),
- et le **plus grand R²**.
---

####  Analyse – Maisons


#####  Meilleur Modèle : `LinearRegression`

- **R² le plus élevé** : 0.022 (seul modèle avec un R² positif)
- **RMSE le plus bas** : 0.930
- Les modèles complexes ne font **pas mieux**, ce qui indique des **données insuffisantes**

##### Interprétation

> Tous les modèles (sauf LinearRegression) ont un **R² négatif**, donc **moins performants que la moyenne**.  
> Cela suggère que les **caractéristiques des maisons sont trop hétérogènes** pour être bien captées sans données supplémentaires.

---

###  Analyse – Appartements



####  Meilleur Modèle : `RandomForest`

- **R² très élevé** : 0.731 → très bon pouvoir prédictif
- **Robuste et performant** sans trop de tuning
- Le modèle optimisé n’apporte pas de gain → les paramètres par défaut sont suffisants

---

###  Comparatif Général : Appartements vs Maisons


- **Appartements** : bonnes performances, modèles efficaces
- **Maisons** : résultats très faibles → manque d'information pour prédire correctement

---

### Recommandations

#### Pour améliorer les performances :

- **Ajouter des variables** : quartier, code postal, année de construction, étage, état du bien, équipements
- Faire du **feature engineering** (création de nouvelles variables dérivées)
- Appliquer une **analyse spatiale** (géolocalisation, proximité)
- Créer des **clusters de maisons** (moderne vs ancien, petite vs grande)

#### Pour les maisons :

- Séparer par type ou style de bien
- Utiliser des méthodes robustes aux faibles volumes (modèles bayésiens, KNN)

---

### Conclusion

- Le modèle **Random Forest** est **très performant sur les appartements** (R² = 0.73)
- Le modèle **LinearRegression est le seul valable pour les maisons**, mais reste faible (R² = 0.02)
- La **qualité des prédictions est directement liée à la richesse des données disponibles**
- Ajouter des données géographiques (quartier, carte)
- Intégrer des caractéristiques précises (année, jardin, état)

>  **Prochaine étape : enrichir les données** pour mieux modéliser les maisons.


### Sauvegarde du modèle global

Objectif : conserver toute l’analyse en un seul bundle (modèles, scalers, résultats)

In [6]:
import pickle
import joblib

# Récupération des meilleurs modèles
best_model_appart = models_appart['RandomForest']
best_model_maison = models_maison['LinearRegression']

# Chargement des scalers sauvegardés
scaler_X_appart = joblib.load('../models/scaler_X_appart.pkl')
scaler_y_appart = joblib.load('../models/scaler_y_appart.pkl')
scaler_X_maison = joblib.load('../models/scaler_X_maison.pkl')
scaler_y_maison = joblib.load('../models/scaler_y_maison.pkl')

# Résultats de validation pour les meilleurs modèles
best_results_appart = resultats_appart['RandomForest']
best_results_maison = resultats_maison['LinearRegression']

# Features utilisées
features = ['Surface reelle bati', 'Nombre de lots', 'Surface terrain'] + list(pd.get_dummies(df['Type local'], drop_first=True).columns)

# Dictionnaire à sauvegarder
lille_bundle = {
    'models_appart': models_appart,  # tous les modèles appartements
    'models_maison': models_maison,  # tous les modèles maisons
    'best_model_appart': best_model_appart,
    'best_model_maison': best_model_maison,
    'scaler_X_appart': scaler_X_appart,
    'scaler_y_appart': scaler_y_appart,
    'scaler_X_maison': scaler_X_maison,
    'scaler_y_maison': scaler_y_maison,
    'features': features,
    'results_appart': resultats_appart,  # tous les résultats appartements
    'results_maison': resultats_maison,  # tous les résultats maisons
    'best_results_appart': best_results_appart,
    'best_results_maison': best_results_maison
}

with open('../models/model_lille.pkl', 'wb') as f:
    pickle.dump(lille_bundle, f)

print("✅ Bundle Lille (tous modèles, scalers, features, résultats, meilleurs modèles) sauvegardé dans : '../models/model_lille.pkl'")

✅ Bundle Lille (tous modèles, scalers, features, résultats, meilleurs modèles) sauvegardé dans : '../models/model_lille.pkl'


### Création d’un bundle allégé avec juste les meilleurs modèles

In [7]:
import pickle
import joblib

# Récupération des meilleurs modèles
best_model_appart = models_appart['RandomForest']
best_model_maison = models_maison['LinearRegression']

# Chargement des scalers sauvegardés
scaler_X_appart = joblib.load('../models/scaler_X_appart.pkl')
scaler_y_appart = joblib.load('../models/scaler_y_appart.pkl')
scaler_X_maison = joblib.load('../models/scaler_X_maison.pkl')
scaler_y_maison = joblib.load('../models/scaler_y_maison.pkl')

# Résultats de validation pour les meilleurs modèles
best_results_appart = resultats_appart['RandomForest']
best_results_maison = resultats_maison['LinearRegression']

# Features utilisées
features = ['Surface reelle bati', 'Nombre de lots', 'Surface terrain'] + list(pd.get_dummies(df['Type local'], drop_first=True).columns)

# Dictionnaire à sauvegarder
lille_bundle = {
    'model_appart': best_model_appart,
    'model_maison': best_model_maison,
    'scaler_X_appart': scaler_X_appart,
    'scaler_y_appart': scaler_y_appart,
    'scaler_X_maison': scaler_X_maison,
    'scaler_y_maison': scaler_y_maison,
    'features': features,
    'results_appart': best_results_appart,
    'results_maison': best_results_maison
}

with open('../models/model_lille_best.pkl', 'wb') as f:
    pickle.dump(lille_bundle, f)

print("✅ Bundle Lille (meilleurs modèles, scalers, features, résultats) sauvegardé dans : '../models/model_lille_best.pkl'")

✅ Bundle Lille (meilleurs modèles, scalers, features, résultats) sauvegardé dans : '../models/model_lille_best.pkl'
